In [1]:
from elasticsearch import Elasticsearch
import pandas as pd
import sqlite3
from datetime import datetime, timezone
from IPython.display import display, HTML
import time
import re

pd.set_option('display.max_rows', None)

db_path = '/Users/zphilipp/git/research/dealsdb/deals_db1.db'
index_name = 'deals' 

In [2]:
es = Elasticsearch(
    ['https://localhost:9200'],
    basic_auth=('elastic', 'iEzoDQe8OWWl133BTsyq'),
    ca_certs='/Users/zphilipp/http_ca.crt'
)

def clean_text(text):
    if text is None:
        return ''
    # HTML
    text = re.sub(r'<[^>]+>', '', text)
    # clean all except letters
    #text = re.sub(r'[^a-zA-Zá-žÁ-Ž0-9\s]', '', text)
    # remove more spaces
    text = re.sub(r'\s+', ' ', text).strip().lower()
    #text = text.replace(["the", "on", ])
    return text

In [3]:
es.info()

ObjectApiResponse({'name': 'bf934ce3043f', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'c7G7Vpm4SOe8Qz5G_y0Otg', 'version': {'number': '8.17.1', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': 'd4b391d925c31d262eb767b8b2db8f398103f909', 'build_date': '2025-01-10T10:08:26.972230187Z', 'build_snapshot': False, 'lucene_version': '9.12.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [4]:
if es.indices.exists(index=index_name):
    es.indices.delete(index=index_name)
else:
    pass
    
index_settings = {
    'settings': {
        'number_of_shards': 1,
        'number_of_replicas': 1 ,
        "max_docvalue_fields_search": 200000
    },
    'mappings': {
        'properties': {
            'id': {'type': 'text'},
            'title_general': {'type': 'text'},
            'option_titles': {'type': 'text'},
            'category' : {'type' : 'text'},
            'timestamp': {'type': 'date'}
        }
    }
}
es.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'deals'})

In [5]:
index_info = es.indices.get(index=index_name)
index_mapping = index_info[index_name]['mappings']
mapping_df = pd.json_normalize(index_mapping['properties'])
mapping_df

search_query = {
  "size": 0
}
response = es.search(index=index_name, body=search_query)
response

ObjectApiResponse({'took': 3, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 0, 'relation': 'eq'}, 'max_score': None, 'hits': []}})

In [6]:
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

def fetch_tree(item_id):
    """Funkce pro načtení stromu pro daný item_id."""
    # Načtěte vlastnosti aktuální položky
    cursor.execute("SELECT parent_id, name FROM category WHERE id = ?", (item_id,))
    item = cursor.fetchone()
    #print(item)
    if not item:
        return None
    
    # Vytvoření uzlu
    return {
        'value': item[1],
        'parent_id': item[0]
    }

def get_taxonomy_path(id):

    data = fetch_tree(id)
    category_string = ""
    while data is not None:    
        data = fetch_tree(data["parent_id"])
        try:
            category_string = data["value"] + " / " + category_string
        except TypeError as e:
            #print (data)
            break
    
    return category_string[:-2]


sql_query = """
    SELECT
        d.deal_id,
        MIN(d.title_general) as title_general,
        GROUP_CONCAT(o.title, ',') AS option_titles,
        MIN(d.customer_category_id) as category_id
    FROM deals d
        LEFT JOIN options o ON (d.deal_id=o.deal_id)
    GROUP BY d.deal_id
"""
cursor.execute(sql_query)
rows = cursor.fetchall()

#cursor.execute("""SELECT d.deal_id, d.title, d.title_general, d.highlights, d.customer_category_id
#   FROM deals d
#""")

counter  = 0
counterTotal = 0
for row in rows:
    if row[3]:
        category = get_taxonomy_path(row[3])
        
        document = {
            'deal_id': row[0],
            'title_general': row[1],
            'option_titles': row[2],
            'category' : category,
            'timestamp': datetime.now(timezone.utc).isoformat()
        }    
        es.index(index=index_name, document=document)
        counter = counter + 1
    counterTotal = counterTotal + 1

conn.close()
print (counter)
print (counterTotal)

93044
105789


In [9]:
#search_term = 'oil change'
search_term = 'Oil changes change Engine replacement services Lubr ication filter Routine oil change Motor oil change Oil maintenance Oil service intervals'
search_term = 'val'
start_time = time.time()
search_query = {
    '_source': ['deal_id', 'title_general', 'option_titles', 'category'],
    'query': {
        'bool': {
            'should': [
                {'match': {'title_general': {'query': search_term, 'boost': 1}}},
                {'match': {'option_titles': {'query': search_term, 'boost': 1}}},
            ]
        }
    },
    'size': 10000
}

response = es.search(index=index_name, body=search_query)
end_time = time.time()
elapsed_time = end_time - start_time

documents = []
for hit in response['hits']['hits']:
    document = hit['_source']
    score = hit['_score']

    document['score'] = score
    documents.append(document)
#print (response)
df = pd.DataFrame(documents)
df['title'] = df.apply(lambda x: f'<a href="https://www.groupon.com/deals/{x["deal_id"]}" target="_blank">{x["title_general"]}</a>', axis=1)
count = df["deal_id"].count()


display(HTML("<strong>Retrieval: " + str(df['deal_id'].count()) + f" deals. In {elapsed_time:.6f}" + "s.</strong>"))
display(HTML(df[["deal_id", "title_general", "category", "score"]].head(1000).to_html(escape=False)))

,deal_id,title_general,category,score
0,viator-mexitours-71,Val 'Quirico - People's Elves Show,Nearby / Things To Do / Sightseeing & Tours,29.234852
1,val-ds-signature,Up to 39% Off on Salon - Hair Care at Val Ds Signature,Nearby / Beauty & Spas / Salons,10.312409
2,beauty-spa-by-val-6,Up to 67% Off on Laser Hair Removal at Beauty spa by val,Nearby / Beauty & Spas / Hair Removal,10.009469
3,skin-haus-by-val,Skin Haus By Val Offers Brazilian Waxing with Optional Relaxing Jelly Mask at Up to 44% Off!,Nearby / Beauty & Spas / Hair Removal,8.956974
4,skin-haus-by-val-1,Skin Haus By Val offers transformative 3-session Procell Micro-Channeling w/ Stem Cell Infusion (Up to 46% Off),Nearby / Beauty & Spas,8.509583
5,hair-by-val-1,"Experience expert men's haircuts with shampoo and style at Hair By Val, offering up to 50% off for ultimate grooming.",Nearby / Beauty & Spas / Hair & Styling,8.302238


### search in categories

In [13]:
query = 'Mattresses'

search_query = {
    'query': {
        'multi_match': {
            'query': query,
            'fields': ['category']
        }
    },
    'size': 10000
}

response = es.search(index=index_name, body=search_query)
documents = []
for hit in response['hits']['hits']:
    document = hit['_source']
    score = hit['_score']
    document['score'] = score
    documents.append(document)
    
df = pd.DataFrame(documents)
df['title'] = df.apply(lambda x: f'<a href="https://www.groupon.com/deals/{x["deal_id"]}" target="_blank">{x["title_general"]}</a>', axis=1)
print(df.count())
display(HTML(df[["title_general", "category", "score"]].head(100).to_html(escape=False)))

deal_id          1
title_general    1
option_titles    1
category         1
timestamp        1
score            1
title            1
dtype: int64


,title_general,category,score
0,Costway 4 Sizes Jacquard 8'' Foam Mattress Medium Firm Bed-in-a-Box,Goods / For the Home / Mattresses & Accessories,9.736887
